Задачи:

* Очистка таблицы от некорректных данных и пропусков

* Построение базовой модели регресии

* Улучшение базового решения с помощью перебора гипперпараметров

* Анализ результатов

In [34]:
import pandas as pd

In [35]:
df = pd.read_csv("16_hw_regression_dataset_final.csv", index_col=0)

In [36]:
df

,Цена,Год,Площадь,Класс,Количество_больниц,Количество_школ,Количество_парков,Количество_метро
id,,,,,,,,
0,5.450380e+06,2009,36.745987,Стандарт,2,3,1,1
1,2.636763e+06,1971,21.731597,Эконом,1,2,0,1
2,5.820842e+06,2001,39.729775,Стандарт,3,2,1,1
3,5.677623e+06,2001,39.529566,Стандарт,1,3,1,1
4,3.080650e+06,1961,27.233553,Эконом,1,1,0,1
...,...,...,...,...,...,...,...,...
2889,1.443585e+07,2018,70.598948,Люкс,7,6,6,3
2890,5.182738e+06,2004,36.513729,Стандарт,2,1,1,1
2891,5.532051e+06,2007,38.096258,Стандарт,3,1,1,1


Предлагается построить модель, которая будет предсказывать **Площадь дома**, основываясь на таких факторах как:

* **Цена дома**
* **Год дома**
* **Класс дома**
* **Количество различной инфраструктуры**

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2894 entries, 0 to 2893
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Цена                2894 non-null   float64
 1   Год                 2894 non-null   int64  
 2   Площадь             2894 non-null   float64
 3   Класс               2894 non-null   object 
 4   Количество_больниц  2894 non-null   int64  
 5   Количество_школ     2894 non-null   int64  
 6   Количество_парков   2894 non-null   int64  
 7   Количество_метро    2894 non-null   int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 203.5+ KB


In [38]:
df.describe()

,Цена,Год,Площадь,Количество_больниц,Количество_школ,Количество_парков,Количество_метро
count,2.894000e+03,2894.000000,2894.000000,2894.000000,2894.000000,2894.000000,2894.000000
mean,8.111370e+06,1998.665860,48.962116,2.863165,3.150657,2.329993,1.491707
std,5.930722e+06,21.889645,26.726675,2.444674,2.138185,2.660529,0.755508
min,7.000000e+00,1950.000000,18.006421,0.000000,1.000000,0.000000,1.000000
25%,3.057700e+06,1981.000000,27.159244,1.000000,1.000000,0.000000,1.000000
50%,5.444229e+06,2005.000000,37.652315,2.000000,2.000000,1.000000,1.000000
75%,1.518529e+07,2018.000000,77.824463,5.000000,5.000000,5.000000,2.000000
max,1.910754e+07,2023.000000,99.933291,7.000000,7.000000,7.000000,3.000000


In [39]:
df['Класс'].unique()

array(['Стандарт', 'Эконом', 'Люкс'], dtype=object)

In [40]:
for i in df['Класс'].unique():
    column_name = 'Класс' + i
    df[column_name] = df['Класс'].isin([i]).astype(int)
df = df.drop(['Класс'], axis = 1 )   
df

,Цена,Год,Площадь,Количество_больниц,Количество_школ,Количество_парков,Количество_метро,КлассСтандарт,КлассЭконом,КлассЛюкс
id,,,,,,,,,,
0,5.450380e+06,2009,36.745987,2,3,1,1,1,0,0
1,2.636763e+06,1971,21.731597,1,2,0,1,0,1,0
2,5.820842e+06,2001,39.729775,3,2,1,1,1,0,0
3,5.677623e+06,2001,39.529566,1,3,1,1,1,0,0
4,3.080650e+06,1961,27.233553,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2889,1.443585e+07,2018,70.598948,7,6,6,3,0,0,1
2890,5.182738e+06,2004,36.513729,2,1,1,1,1,0,0
2891,5.532051e+06,2007,38.096258,3,1,1,1,1,0,0


In [41]:
len_dataset = len(df)

predictor_cols = df.drop(['Цена'],axis = 1).columns
answer_col = 'Цена'

df_train = df.iloc[: int(0.9 * len_dataset)]
df_val = df.iloc[int(0.9 * len_dataset):]

X_train = df_train[predictor_cols]
y_train = df_train[answer_col]


X_val = df_val[predictor_cols]
y_val = df_val[answer_col]


In [42]:
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
neigh.fit(X_train, y_train)

y_prediction = neigh.predict(X_val)

print("MAE =", mean_absolute_error(y_val, y_prediction))

MAE = 377793.010915833


In [43]:
from sklearn.preprocessing import MinMaxScaler

def MinMax_Norm_Sklearn(df_TRAIN, df_VAL, predictors_cols_names):
    
    scaler = MinMaxScaler()

    df_TRAIN[predictors_cols_names] =  scaler.fit_transform(df_train[predictors_cols_names].values)
    df_VAL[predictors_cols_names] =  scaler.transform(df_val[predictors_cols_names].values)
    
    return df_TRAIN, df_VAL

In [44]:
df_train, df_val = MinMax_Norm_Sklearn(df_train, df_val, predictor_cols)

X_train = df_train[predictor_cols]
y_train = df_train[answer_col]


X_val = df_val[predictor_cols]
y_val = df_val[answer_col]


neigh = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
neigh.fit(X_train, y_train)

y_prediction = neigh.predict(X_val)

print("MAE =", mean_absolute_error(y_val, y_prediction))

MAE = 355896.91680426156


C:\Users\Сергей\AppData\Local\Temp\ipykernel_1592\3414204520.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TRAIN[predictors_cols_names] =  scaler.fit_transform(df_train[predictors_cols_names].values)
C:\Users\Сергей\AppData\Local\Temp\ipykernel_1592\3414204520.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_VAL[predictors_cols_names] =  scaler.transform(df_val[predictors_cols_names].values)


In [45]:
def knn_fit_predict_mae(df_TRAIN,  df_VAL, predictors_cols_names, answer_col_name, k, metric):
    
    X_train = df_TRAIN[predictors_cols_names].values
    y_train = df_TRAIN[answer_col_name].values

    X_val = df_VAL[predictors_cols_names].values
    y_val = df_VAL[answer_col_name].values

    neigh = KNeighborsRegressor(n_neighbors=k, metric=metric)
    neigh.fit(X_train, y_train)

    prediction = neigh.predict(X_val)

    return mean_absolute_error(y_val, prediction)

In [46]:
from math import sqrt

k_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, int(sqrt(len(df_train)))]
metric_list = ["euclidean", "manhattan", "chebyshev"]

for k in k_list:
    for metric in metric_list:
        
        mae_k_metric = knn_fit_predict_mae(df_train, df_val, predictor_cols, answer_col, k, metric)
        
        print(mae_k_metric, k, metric)

392830.00112614495 1 euclidean
450420.98186690925 1 manhattan
396774.22378659435 1 chebyshev
388016.3238982031 2 euclidean
418764.85615113605 2 manhattan
389040.40555393894 2 chebyshev
355896.91680426156 3 euclidean
374480.89908488985 3 manhattan
354401.8173618407 3 chebyshev
353362.50422999955 4 euclidean
358922.8816634445 4 manhattan
341885.7179136427 4 chebyshev
348613.60109217116 5 euclidean
352467.12135205045 5 manhattan
377226.53099504515 5 chebyshev
380205.88943628914 6 euclidean
379168.8599794564 6 manhattan
369678.5339127891 6 chebyshev
382531.91842711973 7 euclidean
381266.9199683873 7 manhattan
398098.12429925683 7 chebyshev
383649.21895356127 8 euclidean
379404.489033074 8 manhattan
400877.08469128015 8 chebyshev
381091.0502428913 9 euclidean
388575.8519984536 9 manhattan
397477.35466374387 9 chebyshev
371334.51435300545 10 euclidean
383797.399080315 10 manhattan
396867.7749739346 10 chebyshev
432071.16519586445 51 euclidean
446821.5813215451 51 manhattan
462353.68645995134

In [47]:
from math import sqrt

l = int(sqrt(len(df_train)))
metric_list = ["euclidean", "manhattan", "chebyshev"]
minimum = 1000000
best_mae_k_metric = 0
best_k = 0
best_metric = 0
for k in range(1, int(sqrt(len(df_train)))+1):
    for metric in metric_list:
        mae_k_metric = knn_fit_predict_mae(df_train, df_val, predictor_cols, answer_col, k, metric)
        if mae_k_metric < minimum:
            minimum = mae_k_metric
            best_mae_k_metric = mae_k_metric
            best_k = k
            best_metric = metric
print(best_mae_k_metric, best_k, best_metric)

341885.7179136427 4 chebyshev


In [48]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor


len_dataset = len(df)

predictor_cols = df.drop(['Цена'],axis = 1).columns
answer_col = 'Цена'

df_train = df.iloc[: int(0.9 * len_dataset)]
df_val = df.iloc[int(0.9 * len_dataset):]

df_train, df_val = MinMax_Norm_Sklearn(df_train, df_val, predictor_cols)


X_train = df_train[predictor_cols]
y_train = df_train[answer_col]


X_val = df_val[predictor_cols]
y_val = df_val[answer_col]


neigh = KNeighborsRegressor(n_neighbors=4, metric='chebyshev')

neigh.fit(X_train, y_train)

y_prediction = neigh.predict(X_val)
df_val["Prediction"] = y_prediction

print("MAE =", mean_absolute_error(y_val, y_prediction))

MAE = 341885.7179136427


C:\Users\Сергей\AppData\Local\Temp\ipykernel_1592\3414204520.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TRAIN[predictors_cols_names] =  scaler.fit_transform(df_train[predictors_cols_names].values)
C:\Users\Сергей\AppData\Local\Temp\ipykernel_1592\3414204520.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_VAL[predictors_cols_names] =  scaler.transform(df_val[predictors_cols_names].values)
C:\Users\Сергей\AppData\Local\Temp\ipykernel_1592\3979997853.py:30: SettingWithCopyWarning: 
A value i